In [1]:
import rasterio
from rasterio.transform import rowcol
from pyproj import Transformer

In [2]:
hennepin = rasterio.open("hennepin.tif")
green = rasterio.open("green.tif")

In [3]:
h_to_wgs84 = Transformer.from_crs(hennepin.crs, "EPSG:4326", always_xy=True)
h_from_wgs84 = Transformer.from_crs("EPSG:4326", hennepin.crs, always_xy=True)

g_to_wgs84 = Transformer.from_crs(green.crs, "EPSG:4326", always_xy=True)
g_from_wgs84 = Transformer.from_crs("EPSG:4326", green.crs, always_xy=True)

In [5]:
h_band = hennepin.read(1)
g_band = green.read(1)

First we define the bounding box, which is the same bounding box of the green space band.

Now we write some code that gets a random acre inside the bounding box.

In [ ]:
import numpy as np

ACRE_EDGE_METERS = 63.6149353533
DEGREES_PER_METER = 8.983e-6

In [ ]:
# Get the indices for everything

# Top left for green


In [83]:
from rasterio.windows import Window
green_window = Window(col_off=g_col_min, row_off=g_row_min, width=g_col_max-g_col_min+1, height=g_row_max-g_row_min+1)
heat_window = Window(col_off=h_col_min, row_off=h_row_min, width=h_col_max-h_col_min+1, height=h_row_max-h_row_min+1)


In [84]:
green_data = green.read(1, window=green_window)
heat_data = hennepin.read(1, window=heat_window)

In [85]:
green_data.shape

(7, 8)

In [86]:
heat_data.shape

(7, 6)

In [87]:
green_data.sum(), heat_data.sum()

(np.float32(23.872145), np.float64(3668.7333532301463))

In [ ]:
# Very green area: (np.float32(23.872145), np.float64(8.207601746116067)) OR 3668.7333532301463
# Very red area: (np.float32(0.0), np.float64(8.36240856826096)) OR 4282.99824673542

In [92]:
def generate_point():
    lat = np.random.uniform(
        green.bounds.bottom + ACRE_EDGE_METERS * DEGREES_PER_METER, 
        green.bounds.top
    )

    lon = np.random.uniform(
        green.bounds.left, 
        green.bounds.right - ACRE_EDGE_METERS * DEGREES_PER_METER
    )
    return lat, lon

def generate_window(region, from_wsg84, lon, lat):
    x, y = from_wsg84.transform(lon, lat)
    row_min, col_min = rowcol(region.transform, x, y)

    # Bottom right for green
    x, y = from_wsg84.transform(lon + ACRE_EDGE_METERS * DEGREES_PER_METER, lat - ACRE_EDGE_METERS * DEGREES_PER_METER)
    row_max, col_max = rowcol(region.transform, x, y)
    window = Window(col_off=col_min, row_off=row_min, width=col_max-col_min+1, height=row_max-row_min+1)

    return window

In [ ]:
data = {
    "heat": [],
    "green": []
}

for i in range(10):
    
    # Generate random coordinates
    lat, lon = generate_point()

    # Get windows
    green_window = generate_window(green, g_from_wgs84, lon, lat)
    heat_window = generate_window(hennepin, h_from_wgs84, lon, lat)

    green_data = green.read(1, window=green_window)
    heat_data = hennepin.read(1, window=heat_window)

    if not np.all(np.isfinite(green_data)) or not np.all(np.isfinite(heat_data)):
        continue

    
    data["heat"].append(heat_data.sum())
    data["green"].append(green_data.sum())

In [103]:
data

{'heat': [np.float64(3942.9908205653064),
  np.float64(3756.3204290298263),
  np.float64(4274.721317853295),
  np.float64(3316.8631130250365),
  np.float64(4111.932943353698),
  np.float64(3363.72011425205),
  np.float64(4440.580952835472),
  np.float64(inf),
  np.float64(3104.930930842516),
  np.float64(3563.0168511382712)],
 'green': [np.float32(5.4049587),
  np.float32(10.944492),
  np.float32(11.813987),
  np.float32(2.1824718),
  np.float32(1.5198363),
  np.float32(1.1501467),
  np.float32(4.8887305),
  np.float32(21.845768),
  np.float32(17.066011),
  np.float32(22.388157)]}